<div style="padding:12px 20px; color:white; font-size:1.8em; text-align:center; 
            border-radius:10px; background-color:#2a7f62; font-weight:600; 
            font-family:'Magra', sans-serif; margin:0; overflow:hidden;">
  1. Introduction
</div>

## A Machine Learning (ML) framework for ***CFD simulation on Dry Reforming of Methane (DRM) in an Industrial Scale Reactor*** "

### This code is a part of the research article "https://doi.org/10.1016/j.fuel.2024.132673" Please cite this work if this code was used.
### The code includes Automated Hyperparameter Tuning: Hyperparameter tuning was optimized using the **OPTUNA** library

## # Change below variable according to your problem

### Note: track the word "change" to change the variables according to your problem

In [ ]:
EPOCHS = 100 # No_Epochs ######### Change the number of epochs for model training
optuna_n_trials = 5000   ## Optuna number of trials, for all outputs, change manually if a different value is required by each output

## Notes:
### Search for the "objective_ANN" to change the range of the hyperparameters tuning for each output 
### "epochs": trial.suggest_int("epochs", 100, 1000,100)     ## number of epochs defined manually, but could be tuned...... by adding this line to params inside the objective function

# # Five models are constructed for the five outputs studied

In [ ]:
## File Names for outputs ## Change Name for new model training, otherwise it will load the previously trained model with the same name

# First model (CH4):
model_name_CH4 = "model_Post_CH4_1" 

# Second Model (CO2):
model_name_CO2 = "model_Post_CO2_1"

# Third model (H2)
model_name_H2 = "model_Post_H2_1"

# Fourth Model (CO)
model_name_CO = "model_Post_CO_1"

# Fifth model (C_c)
model_name_C_c = "model_Post_C_c_1"

<div style="padding:12px 20px; color:white; font-size:1.8em; text-align:center; 
            border-radius:10px; background-color:#d46a00; font-weight:600; 
            font-family:'Magra', sans-serif; margin:0; overflow:hidden;">
2) Library Import & Dataset Load
</div>

In [ ]:
# !{sys.executable} -m pip install statsmodels

In [ ]:
import numpy as np  
import pandas as pd  
import matplotlib.pyplot as plt  
import seaborn as sns  

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import optuna
from optuna.trial import TrialState
from optuna.samplers import RandomSampler
from optuna.samplers import CmaEsSampler
from optuna.samplers import TPESampler

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from xgboost import XGBRegressor


# pytorch related imports
import torch
import torch.nn as nn
import torch.optim as optim

import warnings
warnings.filterwarnings('ignore')  # ignore notifications

import sys
import os
from os import path

from datetime import datetime

import shap

## Load dataset file

In [ ]:
data = pd.read_excel(open('Data_ML_Final.xlsx', 'rb'),sheet_name='Final_Data') 

In [ ]:
data

## Data Variables

### Features:
* Reactor type: Fixed or Fluidized (categorical Variable, transformed to One-Hot encoding)  
* Bed Height (m)	
* Superficial Velocity (m/s)
* MFR: (Molar Fraction Ratio) CO2 to CH4 Ratio

### Target variable:
* X_CH4: CH4 conversion rate
* X_CO2: CO2 conversion rate
* Y_H2: H2 Yield
* Y_CO: CO Yield
* C_c: Carbon Content

In [ ]:
data.describe()

In [ ]:
data.corr()['X_CH4'].sort_values() # finding the correlation between the features and the output (CH4 in this case)

<div style="padding:12px 20px; color:white; font-size:1.8em; text-align:center; 
            border-radius:10px; background-color:#333333; font-weight:600; 
            font-family:'Magra', sans-serif; margin:0; overflow:hidden;">
3) Exploratory Data Analysis (EDA)
</div>

As part of the Exploratory Data Analysis (EDA), we will:
* Study of the target feature
* Study features
* Study of the influence of features on the target

In [ ]:
fig = make_subplots(rows=4, cols=1, subplot_titles=('<i>Superficial Velocity (m/s)', '<i>Bed Height (m)',
'<i>MFR', '<i>Fluidized',))

In [ ]:
fig.add_trace(go.Histogram(x=data['Superficial Velocity (m/s)'], name='Superficial Velocity (m/s)'), row=1, col=1)
fig.add_trace(go.Histogram(x=data['Bed Height (m)'], name='Bed Height (m)'), row=2, col=1)
fig.add_trace(go.Histogram(x=data['MFR'], name='MFR'), row=3, col=1)
fig.add_trace(go.Histogram(x=data['Fluidized'], name='Fluidized'), row=4, col=1)

In [ ]:
fig = px.box(data, x="Superficial Velocity (m/s)", y="X_CH4", template='plotly_dark', title='<b>Boxplots', color='Superficial Velocity (m/s)')
fig.update_layout(title_x=0.5)
fig.show()

In [ ]:
fig=px.imshow(data.corr(),text_auto=True, template='plotly_dark', color_continuous_scale=px.colors.sequential.Blues, aspect='auto',title='<b>Correlation matrix')
fig.update_layout(title_x=0.5)
fig.show()

In [ ]:
fig = px.scatter(data, x="MFR", y="X_CO2", trendline="ols", color_discrete_sequence=['steelbl'],
                template='plotly_dark', title='<b>CO2 Conversion dependency on MFR', color='X_CO2', color_continuous_scale=px.colors.sequential.Blues)
fig.update_layout(title_x=0.5)
fig.show()

In [ ]:
fig = px.bar(data.groupby('Fluidized', as_index=False).size().sort_values(by='size', ascending=False), x='Fluidized', y='size',
        color_discrete_sequence=['steelblue'], template='plotly_dark', title='<b> Number of Fluidized Bed values')
fig.update_layout(title_x=0.5)

<div style="padding:12px 20px; color:white; font-size:1.8em; text-align:center; 
            border-radius:10px; background-color:#5b2c6f; font-weight:600; 
            font-family:'Magra', sans-serif; margin:0; overflow:hidden;">
4) Data preprocessing
</div>

In [ ]:
df_inputs = data.drop(['X_CH4', 'X_CO2', 'Y_H2', 'Y_CO', 'C_c', 'Fixed', 'CO2', 'CH4'], axis=1) # dropping the output columns

In [ ]:
df_inputs.head() # check if output columns were dropped

In [ ]:
len(df_inputs) # check length

In [ ]:
## Define Output Matrics (Consider moving at the beginning of the code)
y_CH4 = data['X_CH4']
y_CO2 = data['X_CO2']
y_H2 = data['Y_H2']
y_CO = data['Y_CO']
y_C_c = data['C_c']

## Data Preparation:
1. Splitting Training Dataset (60%), Validation(20%) and Testing Data (20%)
2. Scaling the data (StandardScaler())
3. Transformation for PyTorch

### Note: The above three operations Must be repeated for each output

## 1. For CH4 Model

In [ ]:
# To split data into 3 parts; Train (60%), Validation(20%), and Testing(20%), train_test_split is used twice:
X_trainval_CH4_O, X_test_CH4_O, y_trainval_CH4_O, y_test_CH4_O = train_test_split(df_inputs, y_CH4,test_size=0.2,train_size=0.8, random_state = 42)
X_train_CH4_O, X_valid_CH4_O, y_train_CH4_O, y_valid_CH4_O = train_test_split(X_trainval_CH4_O,y_trainval_CH4_O,test_size = 0.25,train_size =0.75, random_state = 42)

In [ ]:
## SCALE (S) DATA
scaler = StandardScaler()
scaler.fit(X_train_CH4_O)
X_train_CH4_S = scaler.transform(X_train_CH4_O)
X_valid_CH4_S = scaler.transform(X_valid_CH4_O)
X_test_CH4_S = scaler.transform(X_test_CH4_O)

In [ ]:
## Transformation (T) for the pytorch library
X_train_CH4_T = torch.from_numpy(X_train_CH4_S).float()
y_train_CH4_T = torch.from_numpy(y_train_CH4_O.values.reshape(-1,1)).float()
    
X_valid_CH4_T = torch.from_numpy(X_valid_CH4_S).float()
y_valid_CH4_T = torch.from_numpy(y_valid_CH4_O.values.reshape(-1,1)).float()
    
X_test_CH4_T = torch.tensor(X_test_CH4_S).float()
y_test_CH4_T = torch.tensor(y_test_CH4_O.values.reshape(-1,1)).view(-1, 1).float()

## 2. For CO2 Model

In [ ]:
# To split data into 3 parts; Train (60%), Validation(20%), and Testing(20%), train_test_split is used twice:
X_trainval_CO2_O, X_test_CO2_O, y_trainval_CO2_O, y_test_CO2_O = train_test_split(df_inputs, y_CO2,test_size=0.2,train_size=0.8, random_state = 42)
X_train_CO2_O, X_valid_CO2_O, y_train_CO2_O, y_valid_CO2_O = train_test_split(X_trainval_CO2_O,y_trainval_CO2_O,test_size = 0.25,train_size =0.75, random_state = 42)

## SCALE (S) DATA
scaler = StandardScaler()
scaler.fit(X_train_CO2_O)
X_train_CO2_S = scaler.transform(X_train_CO2_O)
X_valid_CO2_S = scaler.transform(X_valid_CO2_O)
X_test_CO2_S = scaler.transform(X_test_CO2_O)

## Transformation (T) for the pytorch library
X_train_CO2_T = torch.from_numpy(X_train_CO2_S).float()
y_train_CO2_T = torch.from_numpy(y_train_CO2_O.values.reshape(-1,1)).float()
    
X_valid_CO2_T = torch.from_numpy(X_valid_CO2_S).float()
y_valid_CO2_T = torch.from_numpy(y_valid_CO2_O.values.reshape(-1,1)).float()
    
X_test_CO2_T = torch.tensor(X_test_CO2_S).float()
y_test_CO2_T = torch.tensor(y_test_CO2_O.values.reshape(-1,1)).view(-1, 1).float()

<div style="padding:12px 20px; color:white; font-size:1.8em; text-align:center; 
            border-radius:10px; background-color:#2a7f62; font-weight:600; 
            font-family:'Magra', sans-serif; margin:0; overflow:hidden;">
5) Model selection, hyperparameter tuning
</div>

In [ ]:
class Model(nn.Module):
    def __init__(self, nfeatures, ntargets, nlayers, hidden_size, dropout, activation_func):
        super().__init__()
        layers = []
        for _ in range(nlayers):
            if len(layers) == 0:
                layers.append(nn.Linear(nfeatures, hidden_size))
                layers.append(nn.Dropout(dropout))
                layers.append(getattr(nn, activation_func)())
            else:
                layers.append(nn.Linear(hidden_size, hidden_size))  ### Same number of nodes for all layers
                layers.append(nn.Dropout(dropout))
                layers.append(getattr(nn, activation_func)() )
        layers.append(nn.Linear(hidden_size, ntargets))
        self.model = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.model(x) 


In [ ]:
class Engine:
    def __init__(self, model, optimizer):
        self.model = model
        self.optimizer = optimizer

    def train(self, X_train, y_train): ## data_loader):
        criterion = nn.MSELoss()
        self.model.train()
        final_loss = 0
        for inputs, targets in zip(X_train, y_train):
            self.optimizer.zero_grad()
            outputs = self.model(inputs)            # forward pass
            loss = criterion(outputs, targets) ## self.loss_fn(targets, outputs)   # defining loss
            loss.backward()                         # computing gradients
            self.optimizer.step()                   # updated weights based on computed gradients
            final_loss += loss.item()               # accumulating running loss
        return final_loss / len(inputs)  ##len(data_loader)
   
    def evaluate(self, X_valid, y_valid):   # data_loader):
        criterion = nn.MSELoss()
        self.model.eval()
        final_loss = 0
        for inputs, targets in zip(X_valid, y_valid):
            outputs = self.model(inputs)
            loss = criterion(outputs, targets)  ## self.loss_fn(targets, outputs)
            final_loss += loss.item()
        return final_loss / len(inputs)  ##len(data_loader)

# run_training is generalized to include all outputs

In [ ]:
def run_training(params, model_name, X_train, X_valid, y_train, y_valid, save_model = False):
    
    ### This model is included in the objective function, but it is used to train the model with the best parameters after optimization ####
    model = Model(
    nfeatures=X_train.shape[1], 
    ntargets=1, # y_train.shape[1]   ## it is defined as 1 since it is the number of outputs for each model
    nlayers=params["num_layers"],         
    hidden_size=params["hidden_size"],     
    dropout=params["dropout"],
    activation_func = params["activation_func"]
             )
    optimizer = getattr(optim, params["optimizer"])(model.parameters(), lr=params["learning_rate"])

    eng = Engine(model, optimizer) 

    best_loss = np.inf
    early_stopping_iter = 10
    early_stopping_counter = 0
    
    for epoch in range(EPOCHS):
        train_loss = eng.train(X_train, y_train) ##train_loader)
        valid_loss = eng.evaluate(X_valid, y_valid) ##train_loader)
        print(f"{epoch}, {train_loss}, {valid_loss}")       
        
        if valid_loss < best_loss:
            best_loss = valid_loss
            if save_model:
                torch.save(model, f"{model_name}.pt" )

        else:
            early_stopping_counter += 1

        if early_stopping_counter > early_stopping_iter:
            break
        
    return best_loss

# Define Functions

## 1. sacatter plot_NN

In [ ]:
def plotscatter_NN(ytrain, ytest, ytrainpred, ytestpred):
    """Plot actual vs. predicted values for training and testing sets from a PyTorch neural network."""
    
    # Convert tensors to NumPy arrays
    ytrain_np = ytrain.view(-1).detach().numpy()
    ytrainpred_np = ytrainpred.view(-1).detach().numpy()
    ytest_np = ytest.view(-1).detach().numpy()
    ytestpred_np = ytestpred.view(-1).detach().numpy()
    
    plt.figure(figsize=(7, 7))

    # Plot training predictions
    plt.plot(ytrain_np, ytrainpred_np, "^", color='violet', markersize=10,
             markeredgecolor='white', markeredgewidth=1)

    # Fit regression line for test predictions
    m, b = np.polyfit(ytest_np, ytestpred_np, 1)

    # Plot test predictions
    plt.plot(ytest_np, ytestpred_np, "^", color='green', markersize=10,
             markeredgecolor='white', markeredgewidth=0.5)

    # Plot regression line for test data
    plt.plot(ytest_np, m * ytest_np + b, color='green', linewidth=2)

    # Labels, legend, and formatting
    plt.legend(["Train Set Prediction", "Test Set Prediction"], fontsize=12, loc='lower right')
    plt.xlabel("Actual Targets", fontsize=15)
    plt.ylabel("Predicted Targets", fontsize=15)
    plt.title("Model Prediction", fontsize=16, weight="bold")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

## 2. model_score

In [ ]:
def model_score(model, X_train, X_valid, X_test, y_train, y_valid, y_test):
    '''prints model scores for training, validation, testing sets for one model (one output)'''   
    ## R2 score
    print('R2 Values:')
    print('                ')
    print(f'Best model R2 in Test: {r2_score(y_test, model(X_test).detach().numpy())}')
    print(f'Best model R2 in Valid: {r2_score(y_valid, model(X_valid).detach().numpy())}')
    print(f'Best model R2 in Train: {r2_score(y_train, model(X_train).detach().numpy())}')
    print('---------------------------------------------')
    
    ## MSE
    print('MSE Values:')
    print('                ')
    print(f'Best model MSE in Test: {mean_squared_error(y_test, model(X_test).detach().numpy())}')
    print(f'Best model MSE in Valid: {mean_squared_error(y_valid, model(X_valid).detach().numpy())}')
    print(f'Best model MSE in Train: {mean_squared_error(y_train, model(X_train).detach().numpy())}')
    print('---------------------------------------------')
    
    ## RMSE
    print('RMSE Values:')
    print('                ')
    print(f'Best model RMSE in Test: {np.sqrt(mean_squared_error(y_test, model(X_test).detach().numpy()))}')
    print(f'Best model RMSE in Valid: {np.sqrt(mean_squared_error(y_valid, model(X_valid).detach().numpy()))}')
    print(f'Best model RMSE in Train: {np.sqrt(mean_squared_error(y_train, model(X_train).detach().numpy()))}')
    print('---------------------------------------------')

# # Train and optimize each output

# For Conversion rates

# 1. First output CH4

## ---- the objective function created with pruning for CH4 (copy below cell for other outputs)-----
### Note: Choose hyperparameter range carefully for better optimization

In [ ]:
"""
Optuna example that optimizes multi-layer perceptrons using PyTorch.

In this example, we optimize the validation accuracy of fashion product recognition using
PyTorch. We optimize the neural network architecture as well as the optimizer
configuration.
"""

### could add a function to choose the output (model name and data), CH4 in this case  #### change for other outputs according to the train_test_split
model_name = model_name_CH4
X_train = X_train_CH4_T 
X_valid = X_valid_CH4_T
X_test = X_test_CH4_T
y_train = y_train_CH4_T
y_valid = y_valid_CH4_T
y_test = y_test_CH4_T
model_path = f'{model_name_CH4}.pt'

def objective_CH4_pruning(trial, save_model = False):
    
    params = {              ## the range of parameters might be too large, thus taking a lot of time 
        "num_layers": trial.suggest_int("num_layers", 1,5,1),   
        "hidden_size": trial.suggest_int("hidden_size", 1, 15,1),  
        "dropout": trial.suggest_uniform("dropout", 0.1, 0.2), 
        "learning_rate": trial.suggest_loguniform("learning_rate",1e-3, 1e-2),
        "optimizer": trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"]),
        "activation_func": trial.suggest_categorical("activation_func", ["ReLU", "Sigmoid", "Tanh"])
#         "epochs": trial.suggest_int("epochs", 100, 1000,100) ## number of epochs defined manually, but could be tuned......
        }

###     def run_training(trial, params, ):   ### training function included in the objective function
    model = Model(
    nfeatures=X_train.shape[1], 
    ntargets=1, # y_train.shape[1]  
    nlayers=params["num_layers"], 
    hidden_size=params["hidden_size"],
    dropout=params["dropout"],
    activation_func = params["activation_func"] 
             )
    optimizer = getattr(optim, params["optimizer"])(model.parameters(), lr=params["learning_rate"]) 
    eng = Engine(model, optimizer) 

    best_loss = np.inf
    early_stopping_iter = 10
    early_stopping_counter = 0
    
    for epoch in range(EPOCHS):
        train_loss = eng.train(X_train, y_train) ##train_loader)
        valid_loss = eng.evaluate(X_valid, y_valid) ##train_loader)
        print(f"{epoch}, {train_loss}, {valid_loss}")
        if valid_loss < best_loss:
            best_loss = valid_loss
            if save_model:
                torch.save(model, f"{model_name}.pt" )
        
        ######## Add pruning ######
        trial.report(best_loss, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
    
        else:
            early_stopping_counter += 1

        if early_stopping_counter > early_stopping_iter:
            break

    print('R2 Values:')
    print('                ')
    print(f'Best model R2 in Test: {r2_score(y_test, model(X_test).detach().numpy())}')
    print(f'Best model R2 in Valid: {r2_score(y_valid, model(X_valid).detach().numpy())}')
    print(f'Best model R2 in Train: {r2_score(y_train, model(X_train).detach().numpy())}')
    print('---------------------------------------------')
        
    return best_loss

### The following command is to run the study with the objective function
if __name__ == "__main__":
    
    if path.isfile(model_path): ## look for the model in the existing directory
        print('Loading pre-trained model from: {}'.format(model_path))
    
    else:
    
        study_ANN = optuna.create_study(direction='minimize', sampler=TPESampler()) 
        study_ANN.optimize(objective_CH4_pruning, n_trials = optuna_n_trials, show_progress_bar = True)   ### change "optuna_n_trials" manually if needed

        ############ Show the best parameters 

        print("Number of finished trials: {}".format(len(study_ANN.trials)))
        # print("Best trial:")
        trial_ANN = study_ANN.best_trial

        print("  Value: {}".format(trial_ANN.value))

        print("  Params: ")
        for key, value in trial_ANN.params.items():
            print("    {}: {}".format(key, value))

        ## Hyperparameter Importance Plot

        fig_1 = optuna.visualization.plot_param_importances(study_ANN)
        fig_1.show()

        ############ Saving the model (trained with the best parameters)    

        print(trial_ANN.params) ## best parameters

        scores = 0
        scr = run_training(trial_ANN.params, model_name, X_train, X_valid, y_train, y_valid, save_model = True)
        print(scr)

        print('Finished training the model. Saving the model to the path: {}'.format(model_path))

        ### Printing Number of Pruned Trials ####

        pruned_trials = study_ANN.get_trials(deepcopy=False, states=[TrialState.PRUNED])
        complete_trials = study_ANN.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

        print("Study statistics: ")
        print("  Number of finished trials: ", len(study_ANN.trials))
        print("  Number of pruned trials: ", len(pruned_trials))
        print("  Number of complete trials: ", len(complete_trials))

0, 0.16955003354883047, 0.006378776888595894
[I 2024-07-14 00:49:27,869] Trial 499 pruned. 
0, 0.22529142218991183, 0.046764464350417256
[I 2024-07-14 00:49:27,926] Trial 500 pruned. 
0, 0.027365299458324444, 0.015295552351744846
[I 2024-07-14 00:49:27,981] Trial 501 pruned. 
0, 0.11138133567658315, 0.001974083545519534
1, 0.030055698974138068, 0.010186087569309166
[I 2024-07-14 00:49:28,047] Trial 502 pruned. 
0, 0.14806595091795316, 0.00375287219094389
[I 2024-07-14 00:49:28,104] Trial 503 pruned. 
0, 0.02705993599897738, 0.007740897111943923
[I 2024-07-14 00:49:28,150] Trial 504 pruned. 
0, 0.06454473512621917, 0.009676608609879622
[I 2024-07-14 00:49:28,206] Trial 505 pruned. 
0, 0.08724078979867045, 0.013432854536404193
[I 2024-07-14 00:49:28,258] Trial 506 pruned. 
0, 0.18842981433772366, 0.045524842833401635
[I 2024-07-14 00:49:28,313] Trial 507 pruned. 
0, 0.03171085379290162, 0.002057438314295723
[I 2024-07-14 00:49:28,368] Trial 508 pruned. 
0, 0.040601577645247744, 0.0053128

0, 0.0795029228867179, 0.0025743525009165324
[I 2024-07-14 00:49:32,707] Trial 572 pruned. 
0, 0.08905857131039774, 0.0012069642239112
1, 0.0459500157849666, 0.0015221126777760219
[I 2024-07-14 00:49:32,790] Trial 573 pruned. 
0, 0.15428496215048426, 0.007488199704539511
[I 2024-07-14 00:49:32,848] Trial 574 pruned. 
0, 0.08236322531502083, 0.02659072814276442
[I 2024-07-14 00:49:32,921] Trial 575 pruned. 
0, 0.26715613469821164, 0.019050061760935932
[I 2024-07-14 00:49:32,989] Trial 576 pruned. 
0, 0.08892874121738714, 0.0029390726340352558
[I 2024-07-14 00:49:33,050] Trial 577 pruned. 
0, 0.0891202316079216, 0.004493260449635272
[I 2024-07-14 00:49:33,120] Trial 578 pruned. 
0, 0.19754048750824893, 0.0017524942304589786
1, 0.021838204861069244, 0.0009430818146825004
2, 0.013748102674753682, 0.0029728760637226515
[I 2024-07-14 00:49:33,226] Trial 579 pruned. 
0, 0.3743405914538016, 0.07022160428459756
[I 2024-07-14 00:49:33,291] Trial 580 pruned. 
0, 0.11046871437793016, 0.00713551972

0, 0.07885963990611344, 0.008160017114022367
[I 2024-07-14 00:49:38,421] Trial 650 pruned. 
0, 0.26549565774757866, 0.0040732366251177154
[I 2024-07-14 00:49:38,482] Trial 651 pruned. 
0, 0.11379422642494319, 0.002884343367895781
[I 2024-07-14 00:49:38,542] Trial 652 pruned. 
0, 0.10089482742250766, 0.009929645704687573
[I 2024-07-14 00:49:38,625] Trial 653 pruned. 
0, 0.10176558826333348, 0.004083139210706577
[I 2024-07-14 00:49:38,703] Trial 654 pruned. 
0, 0.10850870292074433, 0.00268808258624631
[I 2024-07-14 00:49:38,782] Trial 655 pruned. 
0, 0.02720486907492159, 0.005439991829916835
[I 2024-07-14 00:49:38,865] Trial 656 pruned. 
0, 0.03952179941552458, 0.000833160626029894
1, 0.003262535253384158, 0.00048379015061073005
2, 0.0029944357936459998, 0.00037250617339795733
3, 0.007613073545257976, 0.0009018169694172684
4, 0.006106597019254423, 0.00030072750450926833
5, 0.0028926708101835175, 0.0007110265451046871
[I 2024-07-14 00:49:39,016] Trial 657 pruned. 
0, 0.09540580360408057, 

0, 0.08579654114691948, 0.001861171709606424
1, 0.022395038078855123, 0.0003902074995494331
2, 0.021211055951425806, 0.0014216900226529106
3, 0.009317967593919008, 0.0006362020738528429
4, 0.0066978494801333, 0.001855461566918848
[I 2024-07-14 00:49:42,510] Trial 698 pruned. 
0, 0.02758154288312653, 0.004667396508921229
[I 2024-07-14 00:49:42,579] Trial 699 pruned. 
0, 0.08728880491253221, 0.001740970946229936
1, 0.03455288645454857, 0.0008687013878443395
2, 0.012541565714627723, 0.0005363308396226785
3, 0.0120431297798973, 0.0024453109192563716
[I 2024-07-14 00:49:42,685] Trial 700 pruned. 
0, 0.03568422704393015, 0.005996960855554789
[I 2024-07-14 00:49:42,753] Trial 701 pruned. 
0, 0.1554854590880268, 0.00417962557185092
[I 2024-07-14 00:49:42,810] Trial 702 pruned. 
0, 0.10435047598366509, 0.010246813031699276
[I 2024-07-14 00:49:42,870] Trial 703 pruned. 
0, 0.14352597112201693, 0.011491541721625254
[I 2024-07-14 00:49:42,927] Trial 704 pruned. 
0, 0.04186976939308806, 0.002019692

0, 0.0772073911903135, 0.010465257542819018
[I 2024-07-14 00:49:48,061] Trial 770 pruned. 
0, 0.09101987171243309, 0.0031952373938111123
[I 2024-07-14 00:49:48,161] Trial 771 pruned. 
0, 0.03994288213766595, 0.007158039827572793
[I 2024-07-14 00:49:48,252] Trial 772 pruned. 
0, 0.1658722610445693, 0.0024600546480684216
[I 2024-07-14 00:49:48,332] Trial 773 pruned. 
0, 0.03687670403451193, 0.001488768044509925
1, 0.026080347926381364, 0.002551396825055008
[I 2024-07-14 00:49:48,433] Trial 774 pruned. 
0, 0.23121059420645906, 0.005282936530420557
[I 2024-07-14 00:49:48,517] Trial 775 pruned. 
0, 0.053955581262528085, 0.00561481235786232
[I 2024-07-14 00:49:48,605] Trial 776 pruned. 
0, 0.06574523835297441, 0.004242644924488559
[I 2024-07-14 00:49:48,683] Trial 777 pruned. 
0, 0.16641667209478328, 0.0029121915868017823
[I 2024-07-14 00:49:48,762] Trial 778 pruned. 
0, 0.06776330935826991, 0.010498896074182085
[I 2024-07-14 00:49:48,851] Trial 779 pruned. 
0, 0.1071376610444954, 0.00194197

...

0, 0.1326558798853057, 0.0043081692117539205
[I 2024-07-14 01:02:51,407] Trial 4739 pruned. 
0, 0.1533188266068919, 0.005921225991187384
[I 2024-07-14 01:02:51,659] Trial 4740 pruned. 
0, 0.09500294350436889, 0.003948306182827821
[I 2024-07-14 01:02:51,919] Trial 4741 pruned. 
0, 0.1337103266791928, 0.004683930339524522
[I 2024-07-14 01:02:52,181] Trial 4742 pruned. 
0, 0.06181312387889193, 0.006103548521174673
[I 2024-07-14 01:02:52,443] Trial 4743 pruned. 
0, 0.06929223117140282, 0.001043098891614136
1, 0.02480101204218954, 0.0007700634087086655
[I 2024-07-14 01:02:52,710] Trial 4744 pruned. 
0, 0.04207489414329757, 0.006932813934326987
[I 2024-07-14 01:02:52,971] Trial 4745 pruned. 
0, 0.08412135536127607, 0.0013284356573421974
[I 2024-07-14 01:02:53,224] Trial 4746 pruned. 
0, 0.07229820961947553, 0.007591251029225532
[I 2024-07-14 01:02:53,483] Trial 4747 pruned. 
0, 0.06860295176744557, 0.004570998593408149
[I 2024-07-14 01:02:53,745] Trial 4748 pruned. 
0, 0.31598735734587535, 0

0, 0.05014167790886859, 0.01267835013277363
[I 2024-07-14 01:03:17,823] Trial 4827 pruned. 
0, 0.42101273156004027, 0.025285511335823685
[I 2024-07-14 01:03:18,122] Trial 4828 pruned. 
0, 0.053328612535551656, 0.003340791092341533
[I 2024-07-14 01:03:18,442] Trial 4829 pruned. 
0, 0.023247569775094235, 0.009005991453250317
[I 2024-07-14 01:03:18,759] Trial 4830 pruned. 
0, 0.07921323748632858, 0.0034097621246473864
[I 2024-07-14 01:03:19,087] Trial 4831 pruned. 
0, 0.0635939086209305, 0.003633768646977842
[I 2024-07-14 01:03:19,398] Trial 4832 pruned. 
0, 0.10707536920381244, 0.012780169759935234
[I 2024-07-14 01:03:19,708] Trial 4833 pruned. 
0, 0.25468374327320475, 0.00771590271324385
[I 2024-07-14 01:03:20,016] Trial 4834 pruned. 
0, 0.09205371770906368, 0.012458967743441463
[I 2024-07-14 01:03:20,333] Trial 4835 pruned. 
0, 0.03609623929787631, 0.008312099586873956
[I 2024-07-14 01:03:20,663] Trial 4836 pruned. 
0, 0.10244587474517175, 0.0017735815822561563
[I 2024-07-14 01:03:20,9

0, 0.0925464997767449, 0.0016566602698731003
[I 2024-07-14 01:03:41,972] Trial 4904 pruned. 
0, 0.10407522358582355, 0.005534857088861145
[I 2024-07-14 01:03:42,238] Trial 4905 pruned. 
0, 0.18908515566727147, 0.09500666498206556
[I 2024-07-14 01:03:42,498] Trial 4906 pruned. 
0, 0.11614160969838849, 0.0017496138716524001
[I 2024-07-14 01:03:42,776] Trial 4907 pruned. 
0, 0.0418885410472285, 0.003844800251044944
[I 2024-07-14 01:03:43,047] Trial 4908 pruned. 
0, 0.0768754074424578, 0.00973986634198809
[I 2024-07-14 01:03:43,312] Trial 4909 pruned. 
0, 0.08493243298835296, 0.0017943347738764714
[I 2024-07-14 01:03:43,578] Trial 4910 pruned. 
0, 0.039126384110204526, 0.00862102205383053
[I 2024-07-14 01:03:43,841] Trial 4911 pruned. 
0, 0.12127703932128497, 0.01140897589357337
[I 2024-07-14 01:03:44,113] Trial 4912 pruned. 
0, 0.12950402684327855, 0.005816094024226004
[I 2024-07-14 01:03:44,378] Trial 4913 pruned. 
0, 0.046085856918864465, 0.01446144763394841
[I 2024-07-14 01:03:44,647] 

0, 0.1038599101593718, 0.0007438535367327859
1, 0.0071891225870786, 0.00130193654786126
[I 2024-07-14 01:04:05,926] Trial 4989 pruned. 
0, 0.31231290254482147, 0.02493504961603321
[I 2024-07-14 01:04:06,202] Trial 4990 pruned. 
0, 0.07181634259177372, 0.0014013099468783707
[I 2024-07-14 01:04:06,466] Trial 4991 pruned. 
0, 0.06540462277462211, 0.0076654173753922805
[I 2024-07-14 01:04:06,736] Trial 4992 pruned. 
0, 0.19992505223490298, 0.039622251148102805
[I 2024-07-14 01:04:07,002] Trial 4993 pruned. 
0, 0.24480751381906884, 0.006897493760334328
[I 2024-07-14 01:04:07,273] Trial 4994 pruned. 
0, 0.051616191849291226, 0.008789859321950644
[I 2024-07-14 01:04:07,555] Trial 4995 pruned. 
0, 0.09731323537926073, 0.004375691329187248
[I 2024-07-14 01:04:07,833] Trial 4996 pruned. 
0, 0.10250751562853111, 0.00920523535751272
[I 2024-07-14 01:04:08,102] Trial 4997 pruned. 
0, 0.05794580896849766, 0.003960649151849793
[I 2024-07-14 01:04:08,371] Trial 4998 pruned. 
0, 0.08565256157635304, 0.

{'num_layers': 2, 'hidden_size': 15, 'dropout': 0.11090318670640295, 'learning_rate': 0.003746914570923855, 'optimizer': 'RMSprop', 'activation_func': 'ReLU'}
0, 0.12199812311519054, 0.024152085905370768
1, 0.04644063044179347, 0.002769782437098911
2, 0.03029920649714768, 0.0014566382309908477
3, 0.027506483891897915, 0.00416075444809394
4, 0.014558701602567226, 0.003609532649306857
5, 0.018510114428863744, 0.00027620693003971297
6, 0.008601127437657397, 0.0017333530086034443
7, 0.008433847709852671, 0.0019993765615993198
8, 0.01447690728082307, 0.0007780960022500949
9, 0.013069317382523593, 0.000939721149734396
10, 0.007369045348809777, 0.0007707335998929921
11, 0.0036107705555536995, 0.000677420969303455
12, 0.00875501387258737, 0.0014588109061151044
13, 0.00969385170537862, 7.62153591722381e-05
14, 0.006573286097818709, 0.0010231072960777965
15, 0.003941948217971003, 0.0003332147944092867
7.62153591722381e-05
Finished training the model. Saving the model to the path: model_NewInputs

In [ ]:
model_test = torch.load(f'{model_name_CH4}.pt')  ############ already loaded if trained   #   , map_location='cpu')
model_test.eval()
predictions_test = model_test(X_test_CH4_T)

In [ ]:
predictions_test

In [ ]:
model_score(model_test, X_train_CH4_T, X_valid_CH4_T, X_test_CH4_T, y_train_CH4_T, y_valid_CH4_T, y_test_CH4_T)

### Plot CH4 Model Performance

In [ ]:
plotscatter_NN(y_train_CH4_T, y_test_CH4_T, model_test(X_train_CH4_T), model_test(X_test_CH4_T))

# Shap values

In [ ]:
import shap

# Re-initialize JS visualization support
shap.initjs()

# Re-create the explainer (assuming you use DeepExplainer)
explainer = shap.DeepExplainer(model_test, X_train_CH4_T)  # adjust slice as needed

# Recompute SHAP values for the full test set
shap_values = explainer.shap_values(X_test_CH4_T)

# Squeeze SHAP values to remove trailing single dimension
shap_values_clean = shap_values.squeeze(-1)  # shape: (n_samples, n_features)

## Force Plot

In [ ]:
shap.force_plot(
    float(explainer.expected_value[0]),
    shap_values_clean[0].astype(float),
    X_test_CH4_T[0].detach().cpu().numpy().astype(float)
)

## Summary_plot

In [ ]:
shap_values_squeezed = shap_values.squeeze(axis=-1)  # shape becomes (6, 4)
shap.summary_plot(
    shap_values_squeezed,
    X_test_CH4_T.detach().cpu().numpy(),
    feature_names=df_inputs.columns.tolist()
)

In [ ]:
shap.summary_plot(
    shap_values_squeezed,
    X_test_CH4_T.detach().cpu().numpy(),
    feature_names=df_inputs.columns.tolist(),
    plot_type="bar"
)

## Dependence_plot

In [ ]:
feature_names = df_inputs.columns.tolist()

shap.dependence_plot(
    "MFR",                     # Must exactly match one of the feature names
    shap_values,
    X_test_CH4_T.detach().cpu().numpy(),
    feature_names=feature_names     # Required for string-based access
)

In [ ]:
shap.dependence_plot(
    "MFR", shap_values, X_test_CH4_T.cpu().numpy(), feature_names=feature_names, interaction_index="Bed Height (m)"
)

## Decision plot

In [ ]:
shap_values_clean = shap_values.squeeze(-1)  # Now shape = (n_samples, n_features)
shap.decision_plot(
    explainer.expected_value[0],                 # scalar
    shap_values_clean[1],                        # SHAP values for 2nd sample
    feature_names=df_inputs.columns.tolist()     # must match number of features exactly
)

# Find X_CH4 vs. Superficial Velocity Relationship, using the ANN Model

In [ ]:
X_SV = pd.read_excel(open('Data_ML_Final_2.xlsx', 'rb'),sheet_name='SV_outputs') 

In [ ]:
X_SV = X_SV.drop(['Fixed', 'CO2', 'CH4'], axis=1) # dropping the output columns

In [ ]:
X_SV

## prepare the data 
1. Split
2. Scalar
3. transform

In [ ]:
X_train_CH4_scale, X_test_CH4_scale, y_train_CH4_scale, y_test_CH4_scale = train_test_split(df_inputs, y_CH4,test_size=0.2, random_state = 42)

In [ ]:
norm_X_CH4 = StandardScaler().fit(X_train_CH4_scale) # transform training data
X_SV_scaled = norm_X_CH4.transform(X_SV)

In [ ]:
X_SV_scaled = torch.tensor(X_SV_scaled).float()
y_test_CH4_scale = torch.tensor(y_test_CH4_scale.values.reshape(-1,1)).view(-1, 1).float()

In [ ]:
X_CH4_SV = model_test(X_SV_scaled)

## Function to plot the output vs. SV curve for all 4 cases

In [ ]:
def plots_output_SV(X_output_SV):  ## change output 
    
    plt.figure(figsize=(12,8),dpi=200)

    ####### Case 1. Fixed (BH=2.85 m)

    plt.scatter(X_SV['Superficial Velocity (m/s)'][0:13],X_output_SV.detach().numpy()[0:13]) # this is used for the neural network

    #find line of best fit
    a, b = np.polyfit(X_SV['Superficial Velocity (m/s)'][0:13],X_output_SV.detach().numpy()[0:13], 1)
    #add line of best fit to plot
    plt.plot(X_SV['Superficial Velocity (m/s)'][0:13], a*X_SV['Superficial Velocity (m/s)'][0:13]+b, color='blue', linestyle='dashed', linewidth=1, label='Fixed (BH=2.85 m)')


    ####### Case 2. Fludized (BH=2.85 m)

    plt.scatter(X_SV['Superficial Velocity (m/s)'][13:26],X_output_SV.detach().numpy()[13:26])

    #find line of best fit
    a, b = np.polyfit(X_SV['Superficial Velocity (m/s)'][13:26],X_output_SV.detach().numpy()[13:26], 1)
    #add line of best fit to plot
    plt.plot(X_SV['Superficial Velocity (m/s)'][13:26], a*X_SV['Superficial Velocity (m/s)'][13:26]+b, color='green', linestyle='dashed', linewidth=3, label='Fludized (BH=2.85 m)')


    ####### Case 3. Fludized (BH=2.56 5m) (same results as case 2)

    plt.scatter(X_SV['Superficial Velocity (m/s)'][26:39],X_output_SV.detach().numpy()[26:39]) 

    #find line of best fit
    a, b = np.polyfit(X_SV['Superficial Velocity (m/s)'][26:39],X_output_SV.detach().numpy()[26:39], 1)
    #add line of best fit to plot
    plt.plot(X_SV['Superficial Velocity (m/s)'][26:39], a*X_SV['Superficial Velocity (m/s)'][26:39]+b, color='red', linestyle='solid', linewidth=1, label='Fludized (BH=2.56 m)')


    ####### Case 4. Fludized (BH=2.28 m)

    plt.scatter(X_SV['Superficial Velocity (m/s)'][39:52],X_output_SV.detach().numpy()[39:52]) 

    #find line of best fit
    a, b = np.polyfit(X_SV['Superficial Velocity (m/s)'][39:52],X_output_SV.detach().numpy()[39:52], 1)
    #add line of best fit to plot
    plt.plot(X_SV['Superficial Velocity (m/s)'][39:52], a*X_SV['Superficial Velocity (m/s)'][39:52]+b, color='red', linestyle='dashed', linewidth=1, label='Fludized (BH=2.28 m)')

    plt.xlabel('Superficial Velocity (m/s)')
    plt.ylabel('Predicted Output')
    plt.title("Regression plot for NN Model / Output vs. SV (at MFR=1)")

    plt.legend(loc='upper right')
    plt.show()

In [ ]:
plots_output_SV(X_CH4_SV)

# Find X_CH4 vs. MFR Relationship, using the ANN Model

In [ ]:
X_MFR = pd.read_excel(open('Data_ML_Final_2.xlsx', 'rb'),sheet_name='MFR_outputs') 

In [ ]:
X_MFR

In [ ]:
X_MFR = X_MFR.drop(['Fixed', 'CO2', 'CH4'], axis=1) # dropping the output columns

In [ ]:
X_MFR_scaled = norm_X_CH4.transform(X_MFR)
X_MFR_scaled = torch.tensor(X_MFR_scaled).float()
X_CH4_MFR = model_test(X_MFR_scaled)

## Function to plot the output vs. MFR curve for all 4 cases

In [ ]:
def plots_output_MFR(X_output_MFR):  ## change output

    plt.figure(figsize=(12,8),dpi=200)

    ####### Case 1. Fixed (BH=2.85 m)

    plt.scatter(X_MFR['MFR'][0:10],X_output_MFR.detach().numpy()[0:10]) # this is used for the neural network

    #find line of best fit
    a, b = np.polyfit(X_MFR['MFR'][0:10],X_output_MFR.detach().numpy()[0:10], 1)
    #add line of best fit to plot
    plt.plot(X_MFR['MFR'][0:10], a*X_MFR['MFR'][0:10]+b, color='blue', linestyle='dashed', linewidth=1, label='Fixed (BH=2.85 m)')


    ####### Case 2. Fludized (BH=2.85 m)

    plt.scatter(X_MFR['MFR'][10:20],X_output_MFR.detach().numpy()[10:20])

    #find line of best fit
    a, b = np.polyfit(X_MFR['MFR'][10:20],X_output_MFR.detach().numpy()[10:20], 1)
    #add line of best fit to plot
    plt.plot(X_MFR['MFR'][10:20], a*X_MFR['MFR'][10:20]+b, color='green', linestyle='dashed', linewidth=3, label='Fludized (BH=2.85 m)')


    ####### Case 3. Fludized (BH=2.56 5m) (same results as case 2)

    plt.scatter(X_MFR['MFR'][20:30],X_output_MFR.detach().numpy()[20:30]) 

    #find line of best fit
    a, b = np.polyfit(X_MFR['MFR'][20:30],X_output_MFR.detach().numpy()[20:30], 1)
    #add line of best fit to plot
    plt.plot(X_MFR['MFR'][20:30], a*X_MFR['MFR'][20:30]+b, color='red', linestyle='solid', linewidth=1, label='Fludized (BH=2.56 m)')


    ####### Case 4. Fludized (BH=2.28 m)

    plt.scatter(X_MFR['MFR'][30:40],X_output_MFR.detach().numpy()[30:40]) 

    #find line of best fit
    a, b = np.polyfit(X_MFR['MFR'][30:40],X_output_MFR.detach().numpy()[30:40], 1)
    #add line of best fit to plot
    plt.plot(X_MFR['MFR'][30:40], a*X_MFR['MFR'][30:40]+b, color='red', linestyle='dashed', linewidth=1, label='Fludized (BH=2.28 m)')

    plt.xlabel('MFR')
    plt.ylabel('Predicted Output')
    plt.title("Regression plot for NN Model / Output vs. MFR (at SV=0.9)")

    plt.legend(loc='upper left')
    plt.show()

In [ ]:
plots_output_MFR(X_CH4_MFR)

# Find X_CH4 vs. Bed Height Relationship, using the ANN Model

In [ ]:
X_BH = pd.read_excel(open('Data_ML_Final_2.xlsx', 'rb'),sheet_name='BH_outputs') 

In [ ]:
X_BH

In [ ]:
X_BH = X_BH.drop(['Fixed', 'CO2', 'CH4'], axis=1) # dropping the output columns

In [ ]:
X_BH_scaled = norm_X_CH4.transform(X_BH)
X_BH_scaled = torch.tensor(X_BH_scaled).float()

X_CH4_BH = model_test(X_BH_scaled)

In [ ]:
X_CH4_BH

In [ ]:
X_BH['Bed Height (m)'][0:7]

### Function to plot the output vs. BH curve for 2 cases (Fluidized and Fixed)

In [ ]:
def plots_output_BH(X_output_BH):  ## change output

    plt.figure(figsize=(12,8),dpi=200)

    ####### Case 1. Fixed 

    plt.scatter(X_BH['Bed Height (m)'][0:7],X_output_BH.detach().numpy()[0:7]) # this is used for the neural network

    #find line of best fit
    a, b = np.polyfit(X_BH['Bed Height (m)'][0:7],X_output_BH.detach().numpy()[0:7], 1)
    #add line of best fit to plot
    plt.plot(X_BH['Bed Height (m)'][0:7], a*X_BH['Bed Height (m)'][0:7]+b, color='blue', linestyle='dashed', linewidth=1, label='Fixed')

    ####### Case 2. Fludized

    plt.scatter(X_BH['Bed Height (m)'][7:14],X_output_BH.detach().numpy()[7:14]) # this is used for the neural network

    #find line of best fit
    a, b = np.polyfit(X_BH['Bed Height (m)'][7:14],X_output_BH.detach().numpy()[7:14], 1)
    #add line of best fit to plot
    plt.plot(X_BH['Bed Height (m)'][7:14], a*X_BH['Bed Height (m)'][7:14]+b, color='red', linestyle='dashed', linewidth=1, label='Fludized')
    
    plt.xlabel('BH')
    plt.ylabel('Predicted Output')
    plt.title("Regression plot for NN Model / Output vs. BH (at SV=0.9 and MFR = 1)")

    plt.legend(loc='upper left')
    plt.show()

In [ ]:
plots_output_BH(X_CH4_BH)

# 2. Second Output CO2

In [ ]:
"""
Optuna example that optimizes multi-layer perceptrons using PyTorch.

In this example, we optimize the validation accuracy of fashion product recognition using
PyTorch. We optimize the neural network architecture as well as the optimizer
configuration.
"""

### could add function to choose the output (model name and data)
model_name = model_name_CO2
X_train = X_train_CO2_T 
X_valid = X_valid_CO2_T
X_test = X_test_CO2_T
y_train = y_train_CO2_T
y_valid = y_valid_CO2_T
y_test = y_test_CO2_T
model_path = f'{model_name_CO2}.pt'

def objective_CO2_pruning(trial, save_model = False):
    
    params = {              ## the range of parameters might be too large, thus taking a lot of time  (change)
        "num_layers": trial.suggest_int("num_layers", 1,7,1),   
        "hidden_size": trial.suggest_int("hidden_size", 1, 25,1),  
        "dropout": trial.suggest_uniform("dropout", 0.1, 0.3),
        "learning_rate": trial.suggest_loguniform("learning_rate",1e-4, 1e-1),
        "optimizer": trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"]),
        "activation_func": trial.suggest_categorical("activation_func", ["ReLU", "Sigmoid", "Tanh"])
        }
    
###     def run_training(trial, params, ):   ### function included in objective function
    model = Model(
    nfeatures=X_train.shape[1], 
    ntargets=1, # y_train.shape[1]   
    nlayers=params["num_layers"], 
    hidden_size=params["hidden_size"],
    dropout=params["dropout"],
    activation_func = params["activation_func"] 
             )
    optimizer = getattr(optim, params["optimizer"])(model.parameters(), lr=params["learning_rate"]) 
    eng = Engine(model, optimizer) 

    best_loss = np.inf
    early_stopping_iter = 10
    early_stopping_counter = 0
    
    for epoch in range(EPOCHS):
        train_loss = eng.train(X_train, y_train) ##train_loader)
        valid_loss = eng.evaluate(X_valid, y_valid) ##train_loader)
        print(f"{epoch}, {train_loss}, {valid_loss}")
        if valid_loss < best_loss:
            best_loss = valid_loss
            if save_model:
                torch.save(model, f"{model_name}.pt" )
        
        ######## Add pruning ######
        trial.report(best_loss, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
    
        else:
            early_stopping_counter += 1

        if early_stopping_counter > early_stopping_iter:
            break
     
    print('R2 Values:')
    print('                ')
    print(f'Best model R2 in Test: {r2_score(y_test, model(X_test).detach().numpy())}')
    print(f'Best model R2 in Valid: {r2_score(y_valid, model(X_valid).detach().numpy())}')
    print(f'Best model R2 in Train: {r2_score(y_train, model(X_train).detach().numpy())}')
    print('---------------------------------------------')
        
    return best_loss

if __name__ == "__main__":
    
    if path.isfile(model_path): ## look for the model in the existing directory
        print('Loading pre-trained model from: {}'.format(model_path))
    
    else:
    
        study_ANN = optuna.create_study(direction='minimize', sampler=TPESampler()) 
        study_ANN.optimize(objective_CO2_pruning, n_trials = optuna_n_trials, show_progress_bar = True)   ### change "optuna_n_trials" manually if needed

        ############ Show the best parameters 

        print("Number of finished trials: {}".format(len(study_ANN.trials)))
        print("Best trial:")
        trial_ANN = study_ANN.best_trial

        print("  Value: {}".format(trial_ANN.value))

        print("  Params: ")
        for key, value in trial_ANN.params.items():
            print("    {}: {}".format(key, value))

        ## Hyperparameter Importance Plot

        fig_1 = optuna.visualization.plot_param_importances(study_ANN)
        fig_1.show()

        ############ Saving the model (trained with the best parameters)    

        print(trial_ANN.params) ## best parameters

        scores = 0
        scr = run_training(trial_ANN.params, model_name, X_train, X_valid, y_train, y_valid, save_model = True)
        print(scr)

        print('Finished training the model. Saving the model to the path: {}'.format(model_path))

        ### Printing Number of Pruned Trials ####

        pruned_trials = study_ANN.get_trials(deepcopy=False, states=[TrialState.PRUNED])
        complete_trials = study_ANN.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

        print("Study statistics: ")
        print("  Number of finished trials: ", len(study_ANN.trials))
        print("  Number of pruned trials: ", len(pruned_trials))
        print("  Number of complete trials: ", len(complete_trials))

0, 0.5312837820092682, 0.015137744798266795
[I 2024-07-12 23:15:54,960] Trial 503 pruned. 
0, 0.9398368907859549, 0.0956926685757935
[I 2024-07-12 23:15:55,014] Trial 504 pruned. 
0, 0.4660897750291042, 0.025780867552384734
[I 2024-07-12 23:15:55,078] Trial 505 pruned. 
0, 0.03471593362155545, 0.008231798114138655
[I 2024-07-12 23:15:55,151] Trial 506 pruned. 
0, 0.12012946871127372, 0.016980958258500323
[I 2024-07-12 23:15:55,210] Trial 507 pruned. 
0, 0.1701351477594244, 0.0208418644615449
[I 2024-07-12 23:15:55,272] Trial 508 pruned. 
0, 0.07140024586988147, 0.0042815291640181385
[I 2024-07-12 23:15:55,324] Trial 509 pruned. 
0, 0.021291642228788987, 0.00320264179026708
[I 2024-07-12 23:15:55,377] Trial 510 pruned. 
0, 0.018700275722039805, 0.0016498835871061601
[I 2024-07-12 23:15:55,431] Trial 511 pruned. 
0, 0.43251145934846136, 0.09053529817720118
[I 2024-07-12 23:15:55,481] Trial 512 pruned. 
0, 0.07697844411086407, 0.007762290013488382
[I 2024-07-12 23:15:55,534] Trial 513 pru

0, 0.013416729587106602, 0.0014611479864470311
[I 2024-07-12 23:16:00,233] Trial 590 pruned. 
0, 0.09801154394699552, 0.006384418906236533
[I 2024-07-12 23:16:00,316] Trial 591 pruned. 
0, 0.060351442298696156, 0.0018223495571874082
[I 2024-07-12 23:16:00,382] Trial 592 pruned. 
0, 21.932756034308113, 19.789852023124695
[I 2024-07-12 23:16:00,478] Trial 593 pruned. 
0, 0.21736056634108536, 0.004280376580936718
[I 2024-07-12 23:16:00,540] Trial 594 pruned. 
0, 0.055096128534955824, 0.0028157631677458994
[I 2024-07-12 23:16:00,620] Trial 595 pruned. 
0, 0.07568590547987242, 0.012435593549284363
[I 2024-07-12 23:16:00,700] Trial 596 pruned. 
0, 0.2268960285000503, 0.018908166559413075
[I 2024-07-12 23:16:00,776] Trial 597 pruned. 
0, 0.15052478517463896, 0.004012127885971495
[I 2024-07-12 23:16:00,851] Trial 598 pruned. 
0, 0.029829403204303162, 0.004110541496629594
[I 2024-07-12 23:16:00,915] Trial 599 pruned. 
0, 0.04384244007610505, 0.0018422380890115164
[I 2024-07-12 23:16:00,976] Tri

0, 0.0312599677235994, 0.002366709079069551
[I 2024-07-12 23:16:05,327] Trial 659 pruned. 
0, 0.04521602322233775, 0.003439124679971428
[I 2024-07-12 23:16:05,397] Trial 660 pruned. 
0, 0.03177780789837925, 0.0016199212732317392
[I 2024-07-12 23:16:05,466] Trial 661 pruned. 
0, 0.023386089543862454, 0.0025563691724528326
[I 2024-07-12 23:16:05,537] Trial 662 pruned. 
0, 0.31489441730082035, 0.08073921967297792
[I 2024-07-12 23:16:05,612] Trial 663 pruned. 
0, 0.1399566640611738, 0.004415158347001125
[I 2024-07-12 23:16:05,684] Trial 664 pruned. 
0, 0.6079836618155241, 0.18130796775221825
[I 2024-07-12 23:16:05,755] Trial 665 pruned. 
0, 0.019350125512573868, 0.002072400999168167
[I 2024-07-12 23:16:05,832] Trial 666 pruned. 
0, 0.12206322335376285, 0.005026077877118951
[I 2024-07-12 23:16:05,907] Trial 667 pruned. 
0, 0.17138968626659334, 0.01763945755106988
[I 2024-07-12 23:16:05,971] Trial 668 pruned. 
0, 0.023745238912084687, 0.003319206634046168
[I 2024-07-12 23:16:06,043] Trial 66

0, 0.1098070203588577, 0.016726484609534964
[I 2024-07-12 23:16:10,371] Trial 727 pruned. 
0, 0.1038622739870334, 0.0045789325959049165
[I 2024-07-12 23:16:10,450] Trial 728 pruned. 
0, 0.08499862949975068, 0.0021452179134939797
[I 2024-07-12 23:16:10,522] Trial 729 pruned. 
0, 0.021649316715354416, 0.0002809282040061589
1, 0.004106006057575939, 6.262998392969621e-05
2, 0.009169431523446292, 0.0010635061535140267
3, 0.007675913395360112, 0.0004073126747243805
4, 0.009958194910268503, 0.0012416526187735144
5, 0.0047483633950378135, 0.0006974085208639735
6, 0.007143050721523991, 0.00022106030732516047
7, 0.004564436951284279, 0.00032244929889202467
8, 0.003897833401879325, 0.00022335315628652097
9, 0.004223122092547271, 0.000606483565434246
10, 0.004372876642804613, 0.00015887556480365106
R2 Values:
                
Best model R2 in Test: 0.6300960117629699
Best model R2 in Valid: 0.9298447425456041
Best model R2 in Train: 0.818294355870323
---------------------------------------------
[

0, 0.019877459335475578, 0.003021771539351903
[I 2024-07-12 23:16:15,442] Trial 782 pruned. 
0, 0.04359578395724384, 0.0054153458622749895
[I 2024-07-12 23:16:15,535] Trial 783 pruned. 
0, 0.039772523177816765, 0.00017888850373992682
1, 0.013723418571544244, 0.0011602729273363366
2, 0.01269983635893368, 0.0009405460942275568
3, 0.011406436271499842, 0.0012471954632928828
4, 0.00786948336845228, 0.0006796285479140352
5, 0.00988561955273326, 0.0007714324621872493
6, 0.008801114819952716, 0.0002662224765117571
7, 0.008928699125135608, 0.0003008575627312382
8, 0.005716910342755455, 0.0005937289174653415
9, 0.004940404931168985, 0.0003709607916562163
10, 0.010286718877125622, 0.0006457486143744973
R2 Values:
                
Best model R2 in Test: 0.8344756867048315
Best model R2 in Valid: 0.714854976031032
Best model R2 in Train: 0.8131599432089653
---------------------------------------------
[I 2024-07-12 23:16:15,818] Trial 784 finished with value: 0.00017888850373992682 and parameters:

...

0, 0.027863868520398682, 0.0013590028655983133
[I 2024-07-12 23:28:09,010] Trial 4710 pruned. 
0, 0.032116146591192773, 0.0030226157869037706
[I 2024-07-12 23:28:09,284] Trial 4711 pruned. 
0, 0.042749151191401324, 0.003625830839837363
[I 2024-07-12 23:28:09,556] Trial 4712 pruned. 
0, 0.055814214996644296, 0.016770550864748657
[I 2024-07-12 23:28:09,819] Trial 4713 pruned. 
0, 0.10637913887762807, 0.003942331823054701
[I 2024-07-12 23:28:10,093] Trial 4714 pruned. 
0, 0.014940252669930487, 0.000516632504854897
1, 0.0204467995250468, 0.0012172474162071012
[I 2024-07-12 23:28:10,377] Trial 4715 pruned. 
0, 0.08620697724518323, 0.0024460705249111925
[I 2024-07-12 23:28:10,646] Trial 4716 pruned. 
0, 0.06398485750867167, 0.009371155266997278
[I 2024-07-12 23:28:10,907] Trial 4717 pruned. 
0, 0.2188063602152397, 0.02857010898878798
[I 2024-07-12 23:28:11,184] Trial 4718 pruned. 
0, 0.010405389447896596, 0.003949726698920131
[I 2024-07-12 23:28:11,450] Trial 4719 pruned. 
0, 0.0563311654855

0, 0.3934868740689126, 0.07853691873242497
[I 2024-07-12 23:28:34,661] Trial 4797 pruned. 
0, 0.028777645374788108, 0.00206813191425681
[I 2024-07-12 23:28:34,933] Trial 4798 pruned. 
0, 0.0787220753061888, 0.012343456334747316
[I 2024-07-12 23:28:35,206] Trial 4799 pruned. 
0, 0.36637561628594995, 0.13504145946353674
[I 2024-07-12 23:28:35,476] Trial 4800 pruned. 
0, 0.08625510272742076, 0.005819320901764513
[I 2024-07-12 23:28:35,749] Trial 4801 pruned. 
0, 0.01496860948941503, 0.002987851337820757
[I 2024-07-12 23:28:36,025] Trial 4802 pruned. 
0, 0.0203525502292905, 0.0008209992156480439
[I 2024-07-12 23:28:36,294] Trial 4803 pruned. 
0, 0.02781478838210205, 0.001004952342100296
[I 2024-07-12 23:28:36,568] Trial 4804 pruned. 
0, 0.017114219192933433, 0.0009237606187753045
[I 2024-07-12 23:28:36,849] Trial 4805 pruned. 
0, 0.05236836364019837, 0.0023174983325588983
[I 2024-07-12 23:28:37,116] Trial 4806 pruned. 
0, 0.027584870898152758, 0.002413063120457082
[I 2024-07-12 23:28:37,39

0, 0.042264570087354514, 0.0009554215297953306
[I 2024-07-12 23:28:56,642] Trial 4873 pruned. 
0, 0.6817083144560456, 0.07837387593463063
[I 2024-07-12 23:28:56,927] Trial 4874 pruned. 
0, 0.1494500193928161, 0.017929656780324876
[I 2024-07-12 23:28:57,208] Trial 4875 pruned. 
0, 0.02607782572616202, 0.003933285048333346
[I 2024-07-12 23:28:57,486] Trial 4876 pruned. 
0, 0.06706307116837706, 0.0011161275630371392
[I 2024-07-12 23:28:57,764] Trial 4877 pruned. 
0, 0.07614364377332095, 0.0025021203528012848
[I 2024-07-12 23:28:58,037] Trial 4878 pruned. 
0, 0.026067635906656506, 0.003214358248442295
[I 2024-07-12 23:28:58,306] Trial 4879 pruned. 
0, 0.03473066622268561, 0.007803765474818647
[I 2024-07-12 23:28:58,583] Trial 4880 pruned. 
0, 0.018137424856377038, 0.0017359127923555207
[I 2024-07-12 23:28:58,867] Trial 4881 pruned. 
0, 0.04149985763729447, 0.003646656966026285
[I 2024-07-12 23:28:59,147] Trial 4882 pruned. 
0, 0.0311960526077808, 0.0026651718180801254
[I 2024-07-12 23:28:5

0, 0.15190845712254486, 0.0031041121983435005
[I 2024-07-12 23:29:22,578] Trial 4961 pruned. 
0, 0.017584125614575896, 0.0012243065461916558
[I 2024-07-12 23:29:22,860] Trial 4962 pruned. 
0, 0.03608875899260511, 0.002733170687861275
[I 2024-07-12 23:29:23,138] Trial 4963 pruned. 
0, 0.03210713467615278, 0.004885311447651475
[I 2024-07-12 23:29:23,418] Trial 4964 pruned. 
0, 0.0659896010317027, 0.002119462667451444
[I 2024-07-12 23:29:23,704] Trial 4965 pruned. 
0, 0.012533299975984846, 0.004662322116928408
[I 2024-07-12 23:29:23,990] Trial 4966 pruned. 
0, 0.03608250170458405, 0.006644606433837907
[I 2024-07-12 23:29:24,273] Trial 4967 pruned. 
0, 0.05301809493575149, 0.005512424503422153
[I 2024-07-12 23:29:24,552] Trial 4968 pruned. 
0, 0.028578755811849987, 0.002559529094469326
[I 2024-07-12 23:29:24,840] Trial 4969 pruned. 
0, 0.0075874044852222156, 0.0016725686546124052
[I 2024-07-12 23:29:25,128] Trial 4970 pruned. 
0, 0.014077169552138002, 0.0016764085481781876
[I 2024-07-12 23

{'num_layers': 4, 'hidden_size': 20, 'dropout': 0.11664420341042504, 'learning_rate': 0.000734778900274707, 'optimizer': 'RMSprop', 'activation_func': 'Tanh'}
0, 0.03374751751395877, 0.004356384251877898
1, 0.013380262190580083, 0.003727052971953526
2, 0.014827343382876279, 0.0015031352231744677
3, 0.008707135061421178, 0.00045136502285458846
4, 0.013165713244234212, 0.0005785342207218491
5, 0.014303172497193373, 0.0013182726233935682
6, 0.008255383697587604, 0.0005906836986468988
7, 0.015052360670210874, 0.00039671829790677293
8, 0.00901850796162762, 0.00044497737258097914
9, 0.014711163661559112, 0.00017832174482634855
10, 0.007626538990962217, 0.00041512521318054496
11, 0.005797617995483506, 0.0004053410475535202
12, 0.0067109056772096665, 0.00029626795003423467
13, 0.008563347364997753, 0.000554205079993153
14, 0.007131207077691215, 0.0006219796831601343
15, 0.004248167039264672, 0.0003464824799266353
16, 0.005162621661384037, 0.00039578099373116515
0.00017832174482634855
Finished 

In [ ]:
# Best trial:
#   V.....

In [ ]:
model_test_CO2 = torch.load(f'{model_name_CO2}.pt')  ############ already loaded if trained   #   , map_location='cpu')
model_test_CO2.eval()
predictions_test = model_test_CO2(X_test_CO2_T)

In [ ]:
model_score(model_test_CO2, X_train_CO2_T, X_valid_CO2_T, X_test_CO2_T, y_train_CO2_T, y_valid_CO2_T, y_test_CO2_T)

In [ ]:
plotscatter_NN(y_train_CO2_T, y_test_CO2_T, model_test_CO2(X_train_CO2_T), model_test_CO2(X_test_CO2_T))

# Find X_CO2 vs. Superficial Velocity Relationship, using the ANN Model

## prepare the data 
1. Split
2. Scalar
3. transform

In [ ]:
X_train_CO2_scale, X_test_CO2_scale, y_train_CO2_scale, y_test_CO2_scale = train_test_split(df_inputs, y_CO2,test_size=0.2, random_state = 42)

norm_X_CO2 = StandardScaler().fit(X_train_CO2_scale) # transform training data
X_SV_scaled = norm_X_CO2.transform(X_SV)

X_SV_scaled = torch.tensor(X_SV_scaled).float()
y_test_CO2_scale = torch.tensor(y_test_CO2_scale.values.reshape(-1,1)).view(-1, 1).float()

X_CO2_SV = model_test_CO2(X_SV_scaled)

In [ ]:
plots_output_SV(X_CO2_SV)

# Find X_CO2 vs. MFR Relationship, using the ANN Model

In [ ]:
X_MFR_scaled = norm_X_CO2.transform(X_MFR)
X_MFR_scaled = torch.tensor(X_MFR_scaled).float()
X_CO2_MFR = model_test_CO2(X_MFR_scaled)

In [ ]:
plots_output_MFR(X_CO2_MFR)

# For Yields

# 3. For H2

In [ ]:
# To split data into 3 parts; Train (60%), Validation(20%), and Testing(20%), train_test_split is used twice:
X_trainval_H2_O, X_test_H2_O, y_trainval_H2_O, y_test_H2_O = train_test_split(df_inputs, y_H2,test_size=0.2,train_size=0.8, random_state = 42)
X_train_H2_O, X_valid_H2_O, y_train_H2_O, y_valid_H2_O = train_test_split(X_trainval_H2_O,y_trainval_H2_O,test_size = 0.25,train_size =0.75, random_state = 42)

## SCALE (S) DATA
scaler = StandardScaler()
scaler.fit(X_train_H2_O)
X_train_H2_S = scaler.transform(X_train_H2_O)
X_valid_H2_S = scaler.transform(X_valid_H2_O)
X_test_H2_S = scaler.transform(X_test_H2_O)

## Transformation (T) for the pytorch library
X_train_H2_T = torch.from_numpy(X_train_H2_S).float()
y_train_H2_T = torch.from_numpy(y_train_H2_O.values.reshape(-1,1)).float()
    
X_valid_H2_T = torch.from_numpy(X_valid_H2_S).float()
y_valid_H2_T = torch.from_numpy(y_valid_H2_O.values.reshape(-1,1)).float()
    
X_test_H2_T = torch.tensor(X_test_H2_S).float()
y_test_H2_T = torch.tensor(y_test_H2_O.values.reshape(-1,1)).view(-1, 1).float()

In [ ]:
### could add function to choose the output (model name and data) ####### Change for different outputs ############
model_name = model_name_H2
X_train = X_train_H2_T 
X_valid = X_valid_H2_T
y_train = y_train_H2_T
y_valid = y_valid_H2_T
model_path = f'{model_name_H2}.pt'


def objective_H2_pruning(trial, save_model = False):

    params = {              ## the range of parameters might be too large, thus taking a lot of time   (change)
        "num_layers": trial.suggest_int("num_layers", 1,5,1),   
        "hidden_size": trial.suggest_int("hidden_size", 1, 25,1),  
        "dropout": trial.suggest_uniform("dropout", 0.1, 0.2),
        "learning_rate": trial.suggest_loguniform("learning_rate",1e-3, 1e-2),
        "optimizer": trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"]),
        "activation_func": trial.suggest_categorical("activation_func", ["ReLU", "Sigmoid", "Tanh"])
        }
    
###     def run_training(trial, params, ):   ### function included in objective function
    model = Model(
    nfeatures=X_train.shape[1], 
    ntargets=1, # y_train.shape[1]  
    nlayers=params["num_layers"], 
    hidden_size=params["hidden_size"], 
    dropout=params["dropout"],
    activation_func = params["activation_func"] 
             )
    optimizer = getattr(optim, params["optimizer"])(model.parameters(), lr=params["learning_rate"]) 
    eng = Engine(model, optimizer) 

    best_loss = np.inf
    early_stopping_iter = 10
    early_stopping_counter = 0
    
    for epoch in range(EPOCHS):
        train_loss = eng.train(X_train, y_train) ##train_loader)
        valid_loss = eng.evaluate(X_valid, y_valid) ##train_loader)
        print(f"{epoch}, {train_loss}, {valid_loss}")
        if valid_loss < best_loss:
            best_loss = valid_loss
            if save_model:
                torch.save(model, f"{model_name}.pt" )
        
        ######## Add pruning ######
        trial.report(best_loss, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        else:
            early_stopping_counter += 1

        if early_stopping_counter > early_stopping_iter:
            break
            
    return best_loss

############

if __name__ == "__main__":
    
    if path.isfile(model_path): ## look for the model in the existing directory
        print('Loading pre-trained model from: {}'.format(model_path))
    
    else:
    
        study_ANN = optuna.create_study(direction='minimize', sampler=TPESampler()) 
        study_ANN.optimize(objective_H2_pruning, n_trials = optuna_n_trials, show_progress_bar = True)   ### change "optuna_n_trials" manually if needed

        ############ Show the best parameters 

        print("Number of finished trials: {}".format(len(study_ANN.trials)))
        print("Best trial:")
        trial_ANN = study_ANN.best_trial

        print("  Value: {}".format(trial_ANN.value))

        print("  Params: ")
        for key, value in trial_ANN.params.items():
            print("    {}: {}".format(key, value))

        ## Hyperparameter Importance Plot

        fig_1 = optuna.visualization.plot_param_importances(study_ANN)
        fig_1.show()

        ############ Saving the model (trained with the best parameters)    

        print(trial_ANN.params) ## best parameters

        scores = 0
        scr = run_training(trial_ANN.params, model_name, X_train, X_valid, y_train, y_valid, save_model = True)
        print(scr)

        print('Finished training the model. Saving the model to the path: {}'.format(model_path))

        ### Printing Number of Pruned Trials ####

        pruned_trials = study_ANN.get_trials(deepcopy=False, states=[TrialState.PRUNED])
        complete_trials = study_ANN.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

        print("Study statistics: ")
        print("  Number of finished trials: ", len(study_ANN.trials))
        print("  Number of pruned trials: ", len(pruned_trials))
        print("  Number of complete trials: ", len(complete_trials))

In [ ]:
# Best trial:
#   V...

In [ ]:
model_test_H2 = torch.load(f'{model_name_H2}.pt')  ############ already loaded if trained
model_test_H2.eval()
predictions_test = model_test_H2(X_test_H2_T)

In [ ]:
model_score(model_test_H2, X_train_H2_T, X_valid_H2_T, X_test_H2_T, y_train_H2_T, y_valid_H2_T, y_test_H2_T)

In [ ]:
plotscatter_NN(y_train_H2_T, y_test_H2_T, model_test_H2(X_train_H2_T), model_test_H2(X_test_H2_T))

# 4. For CO

In [ ]:
# To split data into 3 parts; Train (60%), Validation(20%), and Testing(20%), train_test_split is used twice:
X_trainval_CO_O, X_test_CO_O, y_trainval_CO_O, y_test_CO_O = train_test_split(df_inputs, y_CO,test_size=0.2,train_size=0.8, random_state = 42)
X_train_CO_O, X_valid_CO_O, y_train_CO_O, y_valid_CO_O = train_test_split(X_trainval_CO_O,y_trainval_CO_O,test_size = 0.25,train_size =0.75, random_state = 42)

## SCALE (S) DATA
scaler = StandardScaler()
scaler.fit(X_train_CO_O)
X_train_CO_S = scaler.transform(X_train_CO_O)
X_valid_CO_S = scaler.transform(X_valid_CO_O)
X_test_CO_S = scaler.transform(X_test_CO_O)

## Transformation (T) for the pytorch library
X_train_CO_T = torch.from_numpy(X_train_CO_S).float()
y_train_CO_T = torch.from_numpy(y_train_CO_O.values.reshape(-1,1)).float()
    
X_valid_CO_T = torch.from_numpy(X_valid_CO_S).float()
y_valid_CO_T = torch.from_numpy(y_valid_CO_O.values.reshape(-1,1)).float()
    
X_test_CO_T = torch.tensor(X_test_CO_S).float()
y_test_CO_T = torch.tensor(y_test_CO_O.values.reshape(-1,1)).view(-1, 1).float()

In [ ]:
### could add function to choose the output (model name and data) ####### Change for different outputs ############
model_name = model_name_CO
X_train = X_train_CO_T 
X_valid = X_valid_CO_T
y_train = y_train_CO_T
y_valid = y_valid_CO_T
model_path = f'{model_name_CO}.pt'

def objective_CO_pruning(trial, save_model = False):
    
    params = {              ## the range of parameters might be too large, thus taking a lot of time  (change)
        "num_layers": trial.suggest_int("num_layers", 1,1,1),   
        "hidden_size": trial.suggest_int("hidden_size", 1, 15,1),  
        "dropout": trial.suggest_uniform("dropout", 0.1, 0.2), 
        "learning_rate": trial.suggest_loguniform("learning_rate",1e-3, 1e-2),
        "optimizer": trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"]),
        "activation_func": trial.suggest_categorical("activation_func", ["ReLU", "Sigmoid", "Tanh"])

        }
###     def run_training(trial, params, ):   ### function included in objective function
    model = Model(
    nfeatures=X_train.shape[1], 
    ntargets=1, # y_train.shape[1]   
    nlayers=params["num_layers"], 
    hidden_size=params["hidden_size"],
    dropout=params["dropout"],
    activation_func = params["activation_func"] 
             )
    optimizer = getattr(optim, params["optimizer"])(model.parameters(), lr=params["learning_rate"]) 
    eng = Engine(model, optimizer) 
    
    best_loss = np.inf
    early_stopping_iter = 10
    early_stopping_counter = 0
    
    for epoch in range(EPOCHS):
        train_loss = eng.train(X_train, y_train) ##train_loader)
        valid_loss = eng.evaluate(X_valid, y_valid) ##train_loader)
        print(f"{epoch}, {train_loss}, {valid_loss}")
        if valid_loss < best_loss:
            best_loss = valid_loss
            if save_model:
                torch.save(model, f"{model_name}.pt" )
        
        ######## Add pruning ######
        trial.report(best_loss, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        else:
            early_stopping_counter += 1

        if early_stopping_counter > early_stopping_iter:
            break
            
    return best_loss

if __name__ == "__main__":
    
    if path.isfile(model_path): ## look for the model in the existing directory
        print('Loading pre-trained model from: {}'.format(model_path))
    
    else:
    
        study_ANN = optuna.create_study(direction='minimize', sampler=TPESampler()) 
        study_ANN.optimize(objective_CO_pruning, n_trials = optuna_n_trials, show_progress_bar = True)    ### change "optuna_n_trials" manually if needed

        ############ Show the best parameters 

        print("Number of finished trials: {}".format(len(study_ANN.trials)))
        print("Best trial:")
        trial_ANN = study_ANN.best_trial

        print("  Value: {}".format(trial_ANN.value))

        print("  Params: ")
        for key, value in trial_ANN.params.items():
            print("    {}: {}".format(key, value))

        ## Hyperparameter Importance Plot

        fig_1 = optuna.visualization.plot_param_importances(study_ANN)
        fig_1.show()

        ############ Saving the model (trained with the best parameters)    

        print(trial_ANN.params) ## best parameters

        scores = 0
        scr = run_training(trial_ANN.params, model_name, X_train, X_valid, y_train, y_valid, save_model = True)
        print(scr)

        print('Finished training the model. Saving the model to the path: {}'.format(model_path))


        ### Printing Number of Pruned Trials ####

        pruned_trials = study_ANN.get_trials(deepcopy=False, states=[TrialState.PRUNED])
        complete_trials = study_ANN.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

        print("Study statistics: ")
        print("  Number of finished trials: ", len(study_ANN.trials))
        print("  Number of pruned trials: ", len(pruned_trials))
        print("  Number of complete trials: ", len(complete_trials))

In [ ]:
# Best trial:
#   V....

In [ ]:
model_test_CO = torch.load(f'{model_name_CO}.pt')  ############ already loaded if trained   #   , map_location='cpu')
model_test_CO.eval()
predictions_test = model_test_CO(X_test_CO_T)

In [ ]:
model_score(model_test_CO, X_train_CO_T, X_valid_CO_T, X_test_CO_T, y_train_CO_T, y_valid_CO_T, y_test_CO2_T)

In [ ]:
plotscatter_NN(y_train_CO_T, y_test_CO_T, model_test_CO(X_train_CO_T), model_test_CO(X_test_CO_T))

# 5. For C_c

In [ ]:
# To split data into 3 parts; Train (60%), Validation(20%), and Testing(20%), train_test_split is used twice:
X_trainval_C_c_O, X_test_C_c_O, y_trainval_C_c_O, y_test_C_c_O = train_test_split(df_inputs, y_C_c,test_size=0.2,train_size=0.8, random_state = 42)
X_train_C_c_O, X_valid_C_c_O, y_train_C_c_O, y_valid_C_c_O = train_test_split(X_trainval_C_c_O,y_trainval_C_c_O,test_size = 0.25,train_size =0.75, random_state = 42)

## SCALE (S) DATA
scaler = StandardScaler()
scaler.fit(X_train_C_c_O)
X_train_C_c_S = scaler.transform(X_train_C_c_O)
X_valid_C_c_S = scaler.transform(X_valid_C_c_O)
X_test_C_c_S = scaler.transform(X_test_C_c_O)

## Transformation (T) for the pytorch library
X_train_C_c_T = torch.from_numpy(X_train_C_c_S).float()
y_train_C_c_T = torch.from_numpy(y_train_C_c_O.values.reshape(-1,1)).float()
    
X_valid_C_c_T = torch.from_numpy(X_valid_C_c_S).float()
y_valid_C_c_T = torch.from_numpy(y_valid_C_c_O.values.reshape(-1,1)).float()
    
X_test_C_c_T = torch.tensor(X_test_C_c_S).float()
y_test_C_c_T = torch.tensor(y_test_C_c_O.values.reshape(-1,1)).view(-1, 1).float()

In [ ]:
### could add function to choose the output (model name and data) ####### Change for different outputs ############
model_name = model_name_C_c
X_train = X_train_C_c_T 
X_valid = X_valid_C_c_T
y_train = y_train_C_c_T
y_valid = y_valid_C_c_T
model_path = f'{model_name_C_c}.pt'


def objective_C_c_pruning(trial, save_model = False):

    params = {              ## the range of parameters might be too large, thus taking a lot of time (change)
        "num_layers": trial.suggest_int("num_layers", 1,7,1),   
        "hidden_size": trial.suggest_int("hidden_size", 1, 25,1),  
        "dropout": trial.suggest_uniform("dropout", 0.1, 0.3), 
        "learning_rate": trial.suggest_loguniform("learning_rate",1e-4, 1e-1),
        "optimizer": trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"]),
        "activation_func": trial.suggest_categorical("activation_func", ["ReLU", "Sigmoid", "Tanh"])

        }
    
###     def run_training(trial, params, ):   ### function included in objective function
    model = Model(
    nfeatures=X_train.shape[1], 
    ntargets=1, # y_train.shape[1]
    nlayers=params["num_layers"],
    hidden_size=params["hidden_size"],
    dropout=params["dropout"],
    activation_func = params["activation_func"] 
             )
    optimizer = getattr(optim, params["optimizer"])(model.parameters(), lr=params["learning_rate"]) 
    eng = Engine(model, optimizer) 
    

    best_loss = np.inf
    early_stopping_iter = 10
    early_stopping_counter = 0
    
    for epoch in range(EPOCHS):
        train_loss = eng.train(X_train, y_train) ##train_loader)
        valid_loss = eng.evaluate(X_valid, y_valid) ##train_loader)
        print(f"{epoch}, {train_loss}, {valid_loss}")
        if valid_loss < best_loss:
            best_loss = valid_loss
            if save_model:
                torch.save(model, f"{model_name}.pt" )
        
        ######## Add pruning ######
        trial.report(best_loss, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        else:
            early_stopping_counter += 1

        if early_stopping_counter > early_stopping_iter:
            break
            
    return best_loss


if __name__ == "__main__":
    
    if path.isfile(model_path): ## look for the model in the existing directory
        print('Loading pre-trained model from: {}'.format(model_path))
    
    else:
    
        study_ANN = optuna.create_study(direction='minimize', sampler=TPESampler()) 
        study_ANN.optimize(objective_C_c_pruning, n_trials = optuna_n_trials, show_progress_bar = True)    ### change "optuna_n_trials" manually if needed

        ############ Show the best parameters 

        print("Number of finished trials: {}".format(len(study_ANN.trials)))
        print("Best trial:")
        trial_ANN = study_ANN.best_trial

        print("  Value: {}".format(trial_ANN.value))

        print("  Params: ")
        for key, value in trial_ANN.params.items():
            print("    {}: {}".format(key, value))

        ## Hyperparameter Importance Plot

        fig_1 = optuna.visualization.plot_param_importances(study_ANN)
        fig_1.show()

        ############ Saving the model (trained with the best parameters)    

        print(trial_ANN.params) ## best parameters

        scores = 0
        scr = run_training(trial_ANN.params, model_name, X_train, X_valid, y_train, y_valid, save_model = True)
        print(scr)

        print('Finished training the model. Saving the model to the path: {}'.format(model_path))


        ### Printing Number of Pruned Trials ####

        pruned_trials = study_ANN.get_trials(deepcopy=False, states=[TrialState.PRUNED])
        complete_trials = study_ANN.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

        print("Study statistics: ")
        print("  Number of finished trials: ", len(study_ANN.trials))
        print("  Number of pruned trials: ", len(pruned_trials))
        print("  Number of complete trials: ", len(complete_trials))

In [ ]:
# Best trial:
#   V.....

In [ ]:
model_test_C_c = torch.load(f'{model_name_C_c}.pt')  ############ already loaded if trained   #   , map_location='cpu')
model_test_C_c.eval()
predictions_test = model_test_C_c(X_test_C_c_T)

In [ ]:
model_score(model_test_C_c, X_train_C_c_T, X_valid_C_c_T, X_test_C_c_T, y_train_C_c_T, y_valid_C_c_T, y_test_C_c_T)

In [ ]:
plotscatter_NN(y_train_C_c_T, y_test_C_c_T, model_test_C_c(X_train_C_c_T), model_test_C_c(X_test_C_c_T))